In [0]:
# DBFS Pfad
DATA_PATH = "workspace.default.yellow_tripdata_2025_01" # "/FileStore/tables/yellow_tripdata_2025_01-1.parquet"
LOOKUP_PATH = "workspace.default.df_lookup"

# DataFrame laden
df_taxi = spark.read.table(DATA_PATH)
df_lookup = spark.read.table(LOOKUP_PATH)

In [0]:
import json
import uuid
import time
import os

# Zielvolume
streaming_input_volume = "/Volumes/workspace/default/taxi_volume/"

df_taxi_small = df_taxi.take(10000)

# Iterator über alle Rows
for i, row in enumerate(df_taxi_small.collect()):
    row_dict = row.asDict()
    
    # eindeutiger Dateiname
    filename = os.path.join(streaming_input_volume, f"event_{i:06d}.json")
    
    # als JSON schreiben
    with open(filename, "w") as f:
        f.write(json.dumps(row_dict))
    
    print(f"Wrote event {i+1}")
    time.sleep(3)  # Pause, um „Streaming“ zu imitieren


In [0]:
import time

streaming_input_volume = "/Volumes/workspace/default/taxi_volume/"

# Assume you already have taxi_df
rows_per_batch = 100
gewichte = 1/rows_per_batch

# Split DF into batches
batches = df_taxi.randomSplit([gewichte]*rows_per_batch, seed=42)  # 500 mini-batches

for i, batch in enumerate(batches):
    write_mode = "overwrite" if i == 0 else "append"
    batch.write.mode(write_mode).format("delta").saveAsTable(f"{streaming_input_volume}/taxi_parquet")
    print(f"Wrote batch {i+1}")
    time.sleep(5)